In [1]:
from sklearn.cluster import KMeans
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('./data/transformed/train_data.csv',parse_dates=['date'],dtype={'link_ID':str})

In [3]:
grp = data[data.month!=7].groupby(['link_ID','time_start_hour'])

In [4]:
c = grp['travel_time'].agg(['mean']).reset_index()

In [5]:
def cluster(data,c,hour):
    c8 = c[c.time_start_hour==hour]
    c8_data = c8.iloc[:,[0,2]].values
    km = KMeans(n_clusters=3,n_jobs=32,max_iter=300)
    x = c8_data[:,1].reshape([-1,1])
    km.fit(x)
    y = km.predict(x)
    c8_label = pd.DataFrame({'link_ID':c8_data[:,0],'c{0}_cluster'.format(hour):y})
    return pd.merge(data,c8_label,how='left',on=['link_ID'])

In [6]:
for i in [8,15,18]:
    data = cluster(data,c,i)

In [7]:
june_missing = ['3377906282328510514', '3377906283328510514']
link_id=data['link_ID'].unique()
sign=[]
for i in link_id:
    if i in june_missing:
        sign.append(1)
    else:
        sign.append(0)
m = pd.DataFrame({'link_ID':link_id,'missing':sign})
data = pd.merge(data,m,how='left',on=['link_ID'])

In [8]:
data.to_csv('./data/transformed/train_data.csv',index=False)